![title](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fdyclassroom.files.wordpress.com%2F2019%2F09%2Fmongodb.jpg&f=1&nofb=1)

<div style="background-color: #168b33; padding: 5px;"></div>

# Unstrukturierte Daten aus einer MongoDB auslesen und auswerten


1. [Übersicht](#Uebersicht)
    1. [Einführung in die dokumentenorientierte MongoDB](#Einführung)
    2. [Ziel dieses Notebooks](#Ziel)
2. [MongoDB Grundlagen](#Grundlagen)
    1. [JSON und BSON](#BSON)
    2. [Relationships with Documents](#Relationships)
    3. [Aufgabenbereich 1](#Aufgabenbereich1)
3. [Starten der MongoDB und Verbdindung herstellen](#Start)
    1. [Lesen, speichern, updaten und löschen von Daten](#CRUD)
    2. [Aufgabenbereich 2](#Aufgabenbereich2)
3. [Betrachten des Datenbestandes](#Datenbestand)
    1. [Importieren eines JSON Datensatzes](#JSON-Einlesen)
    2. [Datensatz mit Pandas anzeigen](#Pandas)
    3. [Die Dokumente direkt auslesen und mit Pandas anzeigen](#Direkt)
    4. [Aufgabenbereich 3](#Aufgabenbereich3)
5. [Erstellen von Queries mit den MongoDB Operatoren](#Queries)
    1. [Aufgabenbereich 4](#Aufgabenbereich4)
6. [Auslesen und Zusammenhänge herausfinden mit der Aggregation Pipeline](#Pipeline)
    1. [Aufgabenbereich 5](#Aufgabenbereich5)
7. [Auslesen und Zusammenhänge herausfinden mit dem Map-Reduce Verfahren](#Map)
    1. [Aufgabenbereich 6](#Aufgabenbereich6)

<div style="background-color: #168b33; padding: 5px; height: 50px;"></div>

## 1. <b id="Uebersicht">Übersicht</b>

### 1.1 <b id="Einführung">Einführung in die dokumentenorientierte MongoDB</b>
Große Datenmengen werden heutzutage schnell generiert. Die MongoDB ist eine dokumentenorientierte Datenbank, die oft dazu verwendet wird um strukturlose Daten in einem verteilten Cluster aus Datenbanken abzuspeichern. Einige Eigenschaften der MongoDB stechen besonders hervor, wenn es um die Wahl des geeigneten Database Management Systems
geht. Hierzu zählen eine richtige Abfragesprache für dynamische Abfragen, eine hohe Verfügbarkeit, Skalierbarkeit, eine effiziente Speicher Engine, Datenkonsistenz und Datenintegrität.

MongoDB eignet sich am besten für große Datenmengen, wenn die vorhandenen Daten weitere Manipulationen für die gewünschte Ausgabe benötigen. Zu den leistungsstarken Funktion zählen hierbei die CRUD-Operationen, das Aggregations-Framework, die Textsuche und die Map-Reduce-Funktion.

### 1.2 Ziel dieses Notebooks
Es soll verdeutlicht werden wie aus einer NoSQL Datenbank Daten effizient ausgelesen werden können, um diese zu bearbeiten. Datensätze liegen nicht immer als CSV Datei vor oder liegen in einem verteilten System aus Datenbanken. Diese Daten direkt aus der Datenbank auszulesen und mit effektiven Werkzeugen wie dem Aggregations-Framework zu strukturieren ist das Ziel dieses Notebooks.

<b>Setup für die Aufgabenüberprüfung</b>

In [ ]:
from taskreview.learningModule import LearningModule
mongo_lm = LearningModule('data/taskReviewDatabase.db')

<div style="background-color: #168b33; padding: 5px; height: 50px;"></div>

## 2. <b id="Grundlagen">Grundlagen</b>

Hierbei sollen die Grundlagen für die Interaktion mit der dokumentenorientierten Datenbank MongoDB und die Verwendung der JavaScript Object Notation (JSON) erklärt werden. Diese ist wichtig, denn die MongoDB speichert Daten nicht in Tabellen, sondern in Collections, welche keine festen Strukturen enthalten. Die Collections können als äquivalent zu den Tabellen in SQL Datenbanken angesehen werden. Die Dokumente selbst liegen innerhalb dieser Collections und werden in einer erweiterten JSON Struktur dargestellt, welche aber mehr Datentypen bietet.

### 2.1 <b id="BSON">JSON und BSON</b>
Der Aufbau der Datensätze wird hier genauer betrachtet. Die Struktur eines einzelnen Dokumentes ist die gleiche wie die von JSON Dateien. In der unteren Abbildung ist ein Dokument zu sehen welches in einer MongoDB gespeichert wurde. Die Daten werden als Key/Value Paare dargestellt, wobei beim ersen Paar mit dem Key "_id" zu erkennen ist, dass der Value einen neuen Datentyp enthällt.

![title](img/image_json.png)

Die neuen Datentypen werden als BSON Datentypen bezeichnet. BSON steht hierbei für Binary JSON. Die MongoDB speichert die Datentypen in einer binären Form ab, wodurch es möglich ist die JSON Datentypen zu ergänzen. Der Key "_id" enthällt somit den BSON Datentyp ObjectId(). Dieses Feld repräsentiert die einzigartige ID jedes Dokumentes. Diese wird benötigt um ein Dokument zu identifizieren (vergleichbar mit Primärschlüssel).

<table style="font-size:14px">
	<tbody>
        <tr>
			<td style="width: 200px;text-align: center;" align="center"><b>BSON Datentypen</b></td>
			<td style="width: 200px;text-align: center;" align="center"><b>JSON Datentypen</b></td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">Double</td>
			<td style="width: 200px;text-align: center;" align="center">Number</td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">String</td>
			<td style="width: 200px;text-align: center;" align="center">String</td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">Object</td>
			<td style="width: 200px;text-align: center;" align="center">Object</td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">Array</td>
			<td style="width: 200px;text-align: center;" align="center">Array</td>
		</tr>
        <tr>
			<td style="width: 200px;text-align: center;" align="center">Boolean</td>
			<td style="width: 200px;text-align: center;" align="center">Boolean</td>
		</tr>
        <tr>
			<td style="width: 200px;text-align: center;" align="center">Null</td>
			<td style="width: 200px;text-align: center;" align="center">Null/Empty</td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">Binary data</td>
			<td style="width: 200px;text-align: center;" align="center"> </td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">Undefined</td>
			<td style="width: 200px;text-align: center;" align="center"> </td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">ObjectId</td>
			<td style="width: 200px;text-align: center;" align="center"> </td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">Date</td>
			<td style="width: 200px;text-align: center;" align="center"> </td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">Regular Expression</td>
			<td style="width: 200px;text-align: center;" align="center"> </td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">JavaScript</td>
			<td style="width: 200px;text-align: center;" align="center"></td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">Symbol</td>
			<td style="width: 200px;text-align: center;" align="center"></td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">32-bit integer</td>
			<td style="width: 200px;text-align: center;" align="center"></td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">64-bit integer</td>
			<td style="width: 200px;text-align: center;" align="center"></td>
		</tr>
	</tbody>
</table>



Im weiteren Schritt wird gezeigt wie die MongoDB gestartet wird und erste Datensätze importiert werden kann. 

Anhand dieser Daten sollen die Grundladen zu den CRUD Operationen verdeutlicht werden. CRUD steht hierbei für die vier Interaktionsmöglichkeiten wenn es darum geht Dokumente zu erstellen (create), lesen (read), aktualisieren (update) und zu löschen (delete).

### 2.2 <b id="Relationships">Relationships with Documents</b>
Die MongoDB kann logische Beziehungen zwischen Dokumenten herleiten. Dazu gibt es verschiedene Verfahren die hier genauer erläutert werden sollen.  

<b>Model One-to-One Relationships with Embedded Documents</b><br>
Ein Dokument kann als Key den Datentyp "Object" enthalten. Dies repräsentiert bei der MongoDB ein Embedded Document. Es wird somit ein gesamgte Dokument unterhalb eines einzigen Keys eingetragen.

![title](img/image_json_embedded.png)

<b>One-to-Many with Document References</b><br>
Ein einzelner Key kann mehrere Schlüssel in einem Array enthalten, welche dann zu vielen weiteren Dokumenten führen.

![title](img/image_json_references.png)

<b>Model One-to-Many Relationships with Embedded Documents</b><br>
In einem Array können auch Objekte als Datentypen eingefügt werden. Ein Schlüssel kann somit viele einzelne Dokumente in einem Array enthalten.

![title](img/image_json_many_embedded.png)

### 2.3 <b id="Aufgabenbereich1">Aufgabenbereich 1</b>

Wie heißt der Datentyp für den Primärschlüssel aus dem Hotel Dokument?

In [ ]:
mongo_lm.show_task(0)

In welcher Form werden die Daten in der MongoDB gespeichert?

In [ ]:
mongo_lm.show_task(1)

In [ ]:
Wie heißt die erweiterte JSON-Form die in der MongoDB verwendet wird? 

In [ ]:
mongo_lm.show_task(2)

Markiere das Embedded Document:

In [ ]:
mongo_lm.show_task(3)

<div style="background-color: #168b33; padding: 5px; height: 50px;"></div>

## 3 <b id="Start">Starten der MongoDB und Verbdindung herstellen</b>

In [ ]:
#!mongod --config /home/jovyan/mongod.conf

<b>MongoDB Treiber importieren</b>

Hier werden alle notwendigen Module für die Interaktion mit der MongoDB und den Daten geladen.

In [ ]:
import pymongo
import datetime
import json 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from bson.son import SON
from bson.objectid import ObjectId
from pprint import pprint

<table style="font-size:14px">
	<tbody >
        <tr>
            <td style="width: 200px;text-align: center;" align="center"><b>Modul</b></td>
            <td style="width: 200px;text-align: center;" align="center"><b>Verwendung</b></td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">pymongo</td>
			<td style="width: 200px;text-align: center;" align="center">MongoDB Python Treiber</td>
		</tr>
        <tr>
			<td style="width: 200px;text-align: center;" align="center">datetime</td>
			<td style="width: 200px;text-align: center;" align="center">Ein neues Datum erstellen</td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">json</td>
			<td style="width: 200px;text-align: center;" align="center">JSON encoder and decoder</td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">matplotlib</td>
			<td style="width: 200px;text-align: center;" align="center">Visualization with Python</td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">pandas</td>
			<td style="width: 200px;text-align: center;" align="center">Daten-Manipulation und -Analyse</td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">numpy</td>
			<td style="width: 200px;text-align: center;" align="center">Mathematische Funktionen</td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">SON (Serialized Ocument Notation)</td>
			<td style="width: 200px;text-align: center;" align="center">Interaktion mit den BSON Documenten</td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">pprint</td>
			<td style="width: 200px;text-align: center;" align="center">Data pretty printer</td>
		</tr>
	</tbody>
</table>

<b>Verbindung zu MongoDB herstellen und alle vorhandenen Datenbanken ausgeben</b>

In [ ]:
client = pymongo.MongoClient()
client.list_database_names()

<b>Eine neue Datenbank und eine Collection anlegen<b>

In [ ]:
mydb = client["Store"]
mycol = mydb["Customer"]

### 3.1 <b id="CRUD">Lesen, speichern, updaten und löschen von Daten</b>


<b>Ein Dokument anlegen und in der Collection abspeichern</b>

In [ ]:
example = {
    "first_name" : "Neuer Name",
    "last_name" : "Neuer Nachname",
    "email" : "irgendwas@irgendwas",
    "phone_number" : "+55 744 995 7575",
    "last_login" : datetime.datetime.now()
}
pprint(example)

In [ ]:
id = mycol.insert_one(example).inserted_id
pprint(id)

<b>Ein Dokument anhand eines Filters auslesen</b>


Zum auslesen eines Dokumentes muss durch das <b>MongoCursor</b> Objekt iteriert werden.<br>
<b style="color:red">Achtung:</b> Wird eine Cursor einmal ausgelesen ist das Object leer.

In [ ]:
filter = {'_id': id}

inserted_Document = mycol.find(filter)
copy_document = inserted_Document.clone()
print(type(inserted_Document))

In [ ]:
for document in inserted_Document:
    pprint(document)

In [ ]:
df = pd.DataFrame(copy_document)
df

<b>Updaten eines Dokumentes anhand eines Filters und dem Set Operator</b>

In [ ]:
filter = {'_id': id} 
newvalue = { "$set": { 'first_name': "Peter Müller" } } 

inserted_Document = mycol.update_one(filter,newvalue)

# Suche nach dem Dokument wiederholen um die Veränderung zu sehen
filter = {'_id': id}
inserted_Document = mycol.find(filter)
copy_document = inserted_Document.clone()

for document in inserted_Document:
    pprint(document)
    
df = pd.DataFrame(copy_document)
df

<b>Nach mehreren Dokumenten suchen</b>

In [ ]:
for document in mycol.find().sort("_id",pymongo.ASCENDING).limit(1):
  pprint(document) 

### 3.2 <b id="Aufgabenbereich2">Aufgabenbereich 2</b>

Was passiert wenn der Cursor ausgegen wurde

In [ ]:
Löschen des Dokumentes welches vorher eingefügt wurde

<div style="background-color: #168b33; padding: 5px; height: 50px;"></div>

## 4. <b id="Datenbestand">Betrachten des Datenbestandes</b>

### 4.1 <b id="JSON-Einlesen">Importieren eines JSON Datensatzes</b>
Die lokalen Daten werden in die MongoDB importiert.

In [ ]:
#!mongoimport --db Hotels --collection booking --file grocery.json --jsonArray

### 4.2 <b id="Pandas">Datensatz mit Pandas anzeigen</b>

Zuerst werden wir die Daten direkt aus dem JSON Dokument, welches lokal in diesem System hinterlegt ist, betrachten. Dieses Dokument wurde aus der MongoDB exportiert und enthällt alle vorhandenen Daten aus der Grocery Collection.

In [ ]:
df = pd.DataFrame(pd.read_json("data/grocery.json", lines = True))
df

Pandas kann den Datensatz nicht korrekt interpretieren. Der Grund dafür ist das die MongoDB die BSON Datentypen im JSON Format als Embedded Document dargestellt werden.
Die MongoDB  ist zwar in der Lage die Daten als CSV Datensatz zu exportieren, aber dafür müssen beim Export Befehl in der Mongo Shell alle notwendigen Felder angegeben werden. Da jedes Dokument eine andere Struktur haben kann muss dieser Befehl abgeändert werden, wenn sich ein einzelnes Dokument um ein Feld vergrößert.

### 4.3 <b id="Direkt">Die Dokumente direkt auslesen und mit Pandas anzeigen</b>

Die Daten die in die MongoDB importiert worden sind, werden nun direkt aus der Collection ausgelesen und angezeigt. Um dies zu ermöglichen wird ein Object der Klassse MongoCursor angelegt welches auf die Dokumente verweist.

In [ ]:
mycol = mydb["Grocery"]
mongo_json = mycol.find({})

In [ ]:
df = pd.DataFrame.from_records(mongo_json)
df

Die Daten der Grocery Collection werden uns Korrekt angezeigt. Im nächsten Schritt wollen wir das gleiche mit den Customer-Daten aus der Customer Collection versuchen.

In [ ]:
mycol = mydb["Customer"]
mongo_json = mycol.find({})
df = pd.DataFrame.from_records(mongo_json)
df

Hierbei ist zu sehen, dass die Embedded Documents nicht als einzelne Spalte interpretiert werden. Diese Daten sollten aber auch korrekt dargestellt werden.

<b>Auslesen der Daten aus dem Embedded Document</b>

In [ ]:
md1 = []
mongo_data1 = {}
keys = ["_id","city", "country", "street","zip"]
for key in keys:
    mongo_json = mycol.find({})
    for doc in mongo_json:
        if key == "_id":
             md1.append(doc["_id"])
        else:
            md1.append(doc["address"].get(key))
    mongo_data1.update({key:list(md1)})
    md1 = []

mongo_dataframe = pd.DataFrame(mongo_data1)
mongo_dataframe

<b>Auslesen der Daten aus einem Array</b>

In [ ]:
md1 = []
mongo_data1 = {}
keys = ["_id","cart"]
for key in keys:
    mongo_json = mycol.find({})
    for doc in mongo_json:
        if key == "_id":
            md1.append(doc["_id"])
        else:
            md1.append(doc["cart"])
    mongo_data1.update({key:list(md1)})
    md1 = []

mongo_dataframe = pd.DataFrame(mongo_data1)
mongo_dataframe

### 4.4 <b id="Aufgabenbereich3">Aufgabenbereich 3</b>

In [ ]:
Warum wird ein Dokument das aus der MongoDB exportiert wurde mit Pandas nicht korrekt angezeigt?

In [ ]:
mongo_lm.show_task(4)

<div style="background-color: #168b33; padding: 5px; height: 50px;"></div>

## 5 <b id="Queries">Erstellen von Queries mit den MongoDB Operatoren</b>

Da wir nun im Kapitel 2 gesehen haben wie mit der MongoDB interagiert werden kann und in Kapiel 3 betrachtet wurde wie die Daten vorliegen, können nun Anfragen an die Datenbank erstellt werden.

Hierzu werden die MongoDB Operatoren verwendet. Diese ermöglichen es spezifische Anfragen zum vergleichen oder aufsuchen von Dokumenten zu implementieren.

<table style="font-size:14px">
	<tbody >
        <tr>
            <td style="width: 200px;text-align: center;" align="center"><b>Operation</b></td>
            <td style="width: 200px;text-align: center;" align="center"><b>Syntax</b></td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">Equality</td>
			<td style="width: 200px;text-align: center;" align="center">\$eq</td>
		</tr>
        <tr>
			<td style="width: 200px;text-align: center;" align="center">Less Than</td>
			<td style="width: 200px;text-align: center;" align="center">\$lt</td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">Values in an array</td>
			<td style="width: 200px;text-align: center;" align="center">\$in</td>
		</tr>
	</tbody>
</table>

In [ ]:
mycol = mydb["Grocery"]

<b>Beispiel:</b> Aus der Collection "Grocery" sollen alle Produkte ausgegeben werden die nicht vorrätig sind.

In [ ]:
cursor = mycol.find({"in_stock" : { "$eq" : False }})
df = pd.DataFrame(cursor)
df

<b>Beispiel:</b> Aus der Collection "Grocery" sollen alle Produkte ausgegeben werden die häufiger als 99x verkauft wurden.

In [ ]:
cursor = mycol.find({"sold":{"$gt":99}})
df = pd.DataFrame(cursor)
df

<b>Beispiel:</b> Aus der Collection "Grocery" sollen alle Produkte ausgegeben werden von dennen weniger als 5 auf Lager sind aber mehr als 3.

In [ ]:
cursor = mycol.find({"$and": [{"stock":{"$gt":3}},{"stock":{"$lt":5}}]})
df = pd.DataFrame(cursor)
df

<b>Beispiel:</b> Aus der Collection "Grocery" und der Collection "Customer" sollen alle Produkte ausgegeben werden die der Kunde mit der ID <b>"5fe60bb2fc13ae64ea000067"</b> gekauft hat.

In [ ]:
mycol = mydb["Customer"]
result = mycol.find({"_id":ObjectId("5fe60bb2fc13ae64ea000067")})
list_of_products = []
for doc in result:
    list_of_products = doc["cart"].copy()
df = pd.DataFrame(list_of_products)
df

In [ ]:
mycol = mydb["Grocery"]
cursor = mycol.find({"_id":{"$in":list_of_products}})
df = pd.DataFrame(cursor)
df

### 5.1 <b id="Aufgabenbereich4">Aufgabenbereich 4</b>

Zeige alle Produkte an die die Kundin mit dem Namen <b>"Eleni Mibourne"</b> eingekauft hat.

In [ ]:
mycol = mydb["Customer"]
result = mycol.find({"first_name" : "Eleni" })
list_of_products = []
for doc in result:
    list_of_products = doc["cart"].copy()

mycol = mydb["Grocery"]
cursor = mycol.find({"_id":{"$in":list_of_products}})
df = pd.DataFrame(cursor)
df

<div style="background-color: #168b33; padding: 5px;height: 50px;"></div>

## 6 <b id="Pipeline">Aggregation Pipeline</b>

Aggregationsoperationen werden verwendet um Ergebnisse zu berechnen, Daten welche in vielen Dokumenten vorliegen unter einem Key zu gruppieren oder um ein Ergebniss in einer seperaten Collection abzuspeichern. Weiterhin können viele Operationen zu einer Pipeline angereiht werden. Die Operationen werden dann auf die bereits gruppierten Ergebnisse angewendet was die Möglichkeiten zur Datenverarbeitung umfassend erweitert.

In [ ]:
mycol = mydb["Customer"]

In [ ]:
pipeline = [
    {"$group": {"_id": "$address.country", "count": {"$sum": 1}}},
    {"$match": {"count": {"$gt": 10}}},
    {"$match": {"_id": {"$regex": '^Po*'}}}
    #{"$sort": SON([("count", -1), ("_id", -1)])}
]

df = pd.DataFrame(mycol.aggregate(pipeline))
df

$trim für bearbeitung von chars?

### 6.1 <b id="Aufgabenbereich5">Aufgabenbereich 5</b>

<div style="background-color: #168b33; padding: 5px;height: 50px;"></div>

## 7 <b id="Map">Map-Reduce</b>

In [ ]:
pipeline = [
    {"$group": {"_id": "$record.ota", "count": {"$sum": 1}}},
     {"$sort": SON([("count", -1), ("_id", -1)])}
]

pprint(list(mycol.aggregate(pipeline)))

### 7.1 <b id="Aufgabenbereich6">Aufgabenbereich 6</b>

for(var x = 950; x<1000;x++){
    var cart = [];
    var groc_rand = Math.floor((Math.random() * 25) + 1);

    for(var i = 0; i<groc_rand;i++){
        var randInt = Math.floor((Math.random() * 1000) + 1);
        var doc = db.Grocery.find().limit(-1).skip(randInt).next();
        cart.push(doc._id);
    }
    cart
    var update_doc = db.Customer.find().skip(x).next();
    db.getCollection('Customer').update(update_doc,{ $set: {"cart": cart} })
}